In [1]:
import pdb
from itertools import product as cartesianproduct

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import RidgeCV

from sklearn.cross_decomposition import PLSRegression as PLS
import pandas_market_calendars as mcal

from valkyrie.data import *
from valkyrie.quants import *
from valkyrie.tools import *
from valkyrie.securities import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib notebook

In [2]:
stocks = stocks_good_dvd()
black_list = {'AGNCP', 'AGNCO', 'NLY PRI'}
stocks = list(filter(lambda x : x not in black_list, stocks))
stocks = [stk for stk in stocks if 'AGNC' in stk]

In [12]:
%%time 
sdate, edate ='2020-06-01', '2021-05-31'
xy_builder = DataMgrCYieldEma(sdate = sdate, edate = edate, stocks = stocks, ema_mults = [5, 15])
model = WinsorizedLM(0.01, PLS, n_components=2)
model = WinsorizedLM(0.01, LR, fit_intercept = False)
df_res = fit_stocks(xy_builder, fit_tscv, [model])
df_res

error loading stocks File d:/valkyrie/data/resampled/hist_ib_raw_md/good_dvd///2020//06//24/AGNCM.h5 does not exist
error loading stocks File d:/valkyrie/data/resampled/hist_ib_raw_md/good_dvd///2020//10//05/AGNCM.h5 does not exist
error loading stocks File d:/valkyrie/data/resampled/hist_ib_raw_md/good_dvd///2020//06//15/AGNCN.h5 does not exist
fitting for AGNCM
fitting h_100_m_Linefit_False_q=0.01_nf_120_nt_10
fitting for AGNCN
fitting h_100_m_Linefit_False_q=0.01_nf_120_nt_10
Wall time: 15.1 s


,param,n_cv,trains,horizon,score_in_median,score_in_mean,score_in_win,score_median,score_mean,score_win
stock,,,,,,,,,,
AGNCM,h_100_m_Linefit_False_q=0.01_nf_120_nt_10,23,120,100,0.065669,0.065669,1.0,0.053576,0.054888,0.913043
AGNCN,h_100_m_Linefit_False_q=0.01_nf_120_nt_10,23,120,100,0.055915,0.055915,1.0,0.061779,0.051069,0.869565


In [7]:
df_res

,param,n_cv,trains,horizon,score_in_median,score_in_mean,score_in_win,score_median,score_mean,score_win
stock,,,,,,,,,,
AGNCM,h_100_m_PLSRn_c_2_q=0.01_nf_120_nt_10,23,120,100,0.052000,0.052000,1.0,0.056490,0.058229,0.956522
AGNCN,h_100_m_PLSRn_c_2_q=0.01_nf_120_nt_10,23,120,100,0.043179,0.043179,1.0,0.047091,0.043791,0.782609


In [ ]:
 df_res, xcols, ycol, wcol = xy_builder.build_xy('AGNCM')

# Naive Test

In [ ]:
df_xy['AGNCM_cy_upx'] = df_xy['AGNCM_cy_upx'].clip(0.065519, 0.078873)

In [ ]:
df_xy = df.copy()
df_xy['AGNCM_cy_upx'] = df_xy['AGNCM_cy_upx'].clip(0.065519, 0.078873)
horizon = 100
ycol = 'AGNCM_cy_upx'
df_xy = df.copy()
y = df_xy.iloc[horizon:] [ycol].values - df_xy.iloc[:-horizon] [ycol].values
x = df_xy.iloc[:-horizon].eval('AGNCM_cy_upx_ema_100 - AGNCM_cy_upx').values
df_xy = df_xy.iloc[:-horizon].copy()
df_xy['x'] = x
df_xy['y'] = y
df_xy['x'] = df_xy['x'].clip(-0.002212, 0.001254)
df_xy['y'] = df_xy['y'].clip(-0.000614, 0.000633)
x = x.reshape(-1,1)

In [ ]:
df_xy.describe(percentiles=[0.01,0.99]).T

In [ ]:
x.shape

In [ ]:
df_zz = pd.DataFrame({
    'r2_in' : re_in,
    'r2_out': re_out})
df_zz

In [ ]:
n_per_day = int( (16 - 9.5 ) * 3600 / 30)
n = 40 * n_per_day
m = 5 *  n_per_day

re_in = []
re_out = []

for i in np.arange((x.shape[0] - n)/m - 1):    
    i = int(i)
    f1,f2 = i * (m+n), (i+1) * n + i * m
    t1,t2 = f2, (i+1) * (n+m)    
    
    lr = LR()
    lr.fit(x[f1:f2], y[f1:f2])
    ri = lr.score(x[f1:f2], y[f1:f2])
    ro = lr.score(x[t1:t2], y[t1:t2])
    re_in.append(ri)
    re_out.append(ro)
    
df_zz = pd.DataFrame({
    'r2_in' : re_in,
    'r2_out': re_out})
df_zz

In [13]:
741.37 - 738.86

2.509999999999991